In [1]:
import pandas as pd
from collections import Counter
import csv

In [2]:
## import Orthogroups with and without 70-15
og_no_70_15 = 'Orthogroups.tsv'
og_w_70_15 = 'Orthogroups_w_70_15.tsv'

In [3]:
og_no_70_15_df = pd.read_csv(og_no_70_15, dtype='string', sep='\t', index_col = 0)
og_w_70_15_df = pd.read_csv(og_w_70_15, dtype='string', sep='\t', index_col = 0)

In [4]:
df_lol = og_no_70_15_df.values.tolist()

og_dict_no_70_15 = {}

for i, row in enumerate(df_lol):
    og = og_no_70_15_df.index[i]
    for cell in row:
        if not pd.isnull(cell):
            for protein in cell.split(', '):
                og_dict_no_70_15[protein] = og

In [5]:
df_lol = og_w_70_15_df.values.tolist()

og_dict_w_70_15 = {}

for i, row in enumerate(df_lol):
    og = og_w_70_15_df.index[i]
    for cell in row:
        if not pd.isnull(cell):
            for protein in cell.split(', '):
                og_dict_w_70_15[protein] = og

In [6]:
genes_per_og_no_70_15 = {}

for gene in og_dict_no_70_15:
    og = og_dict_no_70_15[gene]
    if og not in genes_per_og_no_70_15:
        genes_per_og_no_70_15[og] = []
    genes_per_og_no_70_15[og].append(gene)

In [7]:
genes_per_og_w_70_15 = {}

for gene in og_dict_w_70_15:
    og = og_dict_w_70_15[gene]
    if og not in genes_per_og_w_70_15:
        genes_per_og_w_70_15[og] = []
    genes_per_og_w_70_15[og].append(gene)

In [52]:
## make translation dictionary
translation_w_70_15_to_no_70_15 = {}

for og in genes_per_og_w_70_15:
    proteins_in_og = genes_per_og_w_70_15[og]
    ogs_no_70_15 = []
    for protein in proteins_in_og:
        if 'MGG' not in protein:
            try:
                ogs_no_70_15.append(og_dict_no_70_15[protein])
            except: ## a few proteins don't end up in ogs
                pass
    if len(ogs_no_70_15) > 0:
        c = Counter(ogs_no_70_15)
        og_translated = c.most_common(1)[0][0]
        translation_w_70_15_to_no_70_15[og] = og_translated

In [8]:
## probably need to do something better than just the best BUSCO annotation
pannzer_out = 'MG8_pannzer_5_12_22.txt'
pannzer_df = pd.read_csv(pannzer_out, dtype='string', sep='\t', index_col = 0)

In [9]:
# remove argot_rank less than 1
pannzer_df = pannzer_df[pannzer_df['ARGOT_rank'] == '1' ]

In [13]:
pannzer_df

,ontology,goid,desc,ARGOT_score,ARGOT_PPV,ARGOT_rank,goclasscount
qpid,,,,,,,
MGG_07573T0,MF,0004198,calcium-dependent cysteine-type endopeptidase ...,12.2947044293,0.813826453125,1,84
MGG_07573T0,BP,0006508,proteolysis,4.20609576672,0.602487315535,1,84
MGG_05134T0,BP,0001732,formation of cytoplasmic translation initiatio...,11.3014173894,0.792827273237,1,91
MGG_05134T0,CC,0016282,eukaryotic 43S preinitiation complex,11.0417005466,0.787185869525,1,91
MGG_05134T0,MF,0003743,translation initiation factor activity,8.53339611501,0.728857908837,1,95
...,...,...,...,...,...,...,...
MGG_06165T0,BP,0006021,inositol biosynthetic process,11.8622938739,0.804793207548,1,90
MGG_06165T0,MF,0008934,inositol monophosphate 1-phosphatase activity,11.7231926077,0.801852430021,1,93
MGG_06713T0,CC,0000775,"chromosome, centromeric region",9.81715912008,0.759645665344,1,2


In [14]:
n_ogs_with_multiple_mggs = []
n_ogs_with_matching_go = []
n_ogs_with_mismatched_go = []

for og in genes_per_og_w_70_15:
    mggs = []
    for gene in genes_per_og_w_70_15[og]:
        if 'MGG' in gene:
            mggs.append(gene)
    mggs_goids_lol = []
    if len(mggs) > 1:
        for mgg in mggs:
            if mgg in pannzer_df.index: ## check if mgg is in the column
                if isinstance((pannzer_df.loc[mgg]['goid']), str): ## sometimes tolist() doesnt work bc there is only one
                    mggs_goids_lol.append([pannzer_df.loc[mgg]['goid']])
                else:
                    mggs_goids_lol.append(sorted(pannzer_df.loc[mgg]['goid'].tolist()))
    if mggs_goids_lol: # if it isn't empty
        n_ogs_with_multiple_mggs.append(og)
        if mggs_goids_lol.count(mggs_goids_lol[0]) == len(mggs_goids_lol):
            n_ogs_with_matching_go.append(og)
        else:
            n_ogs_with_mismatched_go.append(og)
            print('some differences')
            print(mggs_goids_lol)

some differences
[['0003924', '0005874'], ['0003924']]
some differences
[['0005576', '0044275', '0050525'], ['0005576', '0050525'], ['0005576', '0044275', '0050525']]
some differences
[['0016887'], ['0016021', '0016705']]
some differences
[['0006508', '0008236'], ['0044281']]
some differences
[['0003824', '0044281'], ['0047834']]
some differences
[['0016021'], ['0016021', '0055085']]
some differences
[['0015693', '0016021', '0046873'], ['0016021']]
some differences
[['0004672', '0006468', '0016021'], ['0015693', '0016021', '0046873']]
some differences
[['0050660'], ['0005938', '0044550', '0050660'], ['0016021', '0050660']]
some differences
[['0009058', '0031177'], ['0016746']]
some differences
[['0004402', '0005634', '0016573'], ['0016021', '0022857', '0055085']]
some differences
[['0016021'], ['0004252', '0006508']]
some differences
[['0005789', '0097502', '0102704'], ['0000124', '0003712', '2000112']]
some differences
[['0000981', '0005634', '0006357'], ['0031965', '0055088']]
some d

In [15]:
## very few conflicts to resolve in general
print(len(n_ogs_with_multiple_mggs))
print(len(n_ogs_with_matching_go))
print(len(n_ogs_with_mismatched_go))

193
160
33


In [16]:
## assign go ids to 70-15 ogs
go_ids_per_og = {}

for og in genes_per_og_w_70_15:
    mggs = []
    for gene in genes_per_og_w_70_15[og]:
        if 'MGG' in gene:
            mggs.append(gene)
    if len(mggs) > 1:
        mggs_goids_lol = []
        for mgg in mggs:
            if mgg in pannzer_df.index: ## check if mgg is in the df
                if isinstance((pannzer_df.loc[mgg]['goid']), str): ## sometimes tolist() doesnt work bc there is only one
                    mggs_goids_lol.append([pannzer_df.loc[mgg]['goid']])
                else:
                    mggs_goids_lol.append(sorted(pannzer_df.loc[mgg]['goid'].tolist()))
        if mggs_goids_lol: # if it isn't empty
            if mggs_goids_lol.count(mggs_goids_lol[0]) == len(mggs_goids_lol): # if the GO IDs are all the same
                go_ids_per_og[og] = mggs_goids_lol[0]
            else:
                go_ids = []
                for ont in ['MF', 'BP', 'CC']:
                    subset_df = pannzer_df[(pannzer_df['ontology'] == ont) & (pannzer_df.index.isin(mggs))]
                    if not subset_df.empty: # some onts are empty
                        c = Counter(subset_df['goid'].tolist())
                        if len(c) != len(mggs): ## if one appears more than once pick the max
                            go_id = max(c, key=c.get)
                        else:
                            go_id = subset_df.sort_values('ARGOT_PPV').iloc[-1]['goid']
                    go_ids.append(go_id)
                go_ids_per_og[og] = go_ids
    elif len(mggs) == 1 and mggs[0] in pannzer_df.index: # if just one MGG in orthogroup just use those GO IDs
        if isinstance((pannzer_df.loc[mggs[0]]['goid']), str): ## sometimes tolist() doesnt work bc there is only one
            go_ids_per_og[og] = [pannzer_df.loc[mggs[0]]['goid']]
        else:
            go_ids_per_og[og] = sorted(pannzer_df.loc[mggs[0]]['goid'].tolist())

In [67]:
go_ids_per_og_translated = {}

for og in go_ids_per_og:
    go_terms = go_ids_per_og[og]
    try:
        og_translated = translation_w_70_15_to_no_70_15[og]
    except: ## some ogs dont line up
        continue
    go_ids_per_og_translated[og_translated] = go_terms

In [22]:
# print annotations
with open('Orthogroups_GO_ids.txt', 'w', newline = '') as output_csv:
    w = csv.writer(output_csv, delimiter = '\t')
    for og in go_ids_per_og:
        if len(go_ids_per_og[og]) == 1:
            ids = 'GO:' + go_ids_per_og[og][0]
        else:
            ids = []
            for go_id in go_ids_per_og[og]:
                ids.append('GO:'+go_id+', ')
            ids = ''.join(ids)
            ids = ids[:-2]
        w.writerow([og, ids])